In [26]:
import os
import nibabel as nib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from dipy.reconst.dti import TensorModel, fractional_anisotropy, mean_diffusivity, radial_diffusivity, axial_diffusivity
from dipy.io.image import load_nifti_data
from dipy.core.gradients import gradient_table

In [27]:
data_dir = "/media/ist/Drive2/MANSOOR/Neuroimaging-Project/DTI-Analysis/Data/test"

In [30]:
# set the file paths
data_files = os.path.join(data_dir, 'sherbrooke_3shell')
bvals_file = os.path.join(data_files, 'HARDI193.bval') 
bvecs_file = os.path.join(data_files, 'HARDI193.bvec')
img_file = os.path.join(data_files, 'HARDI193.nii.gz')  
atlas_file = os.path.join(data_dir, 'AAL.nii.gz')  

# Load data
img = load_nifti_data(img_file)
bvals = np.loadtxt(bvals_file)
bvecs = np.loadtxt(bvecs_file)
atlas = load_nifti_data(atlas_file)
print(img.shape, bvals.shape, bvecs.shape, atlas.shape)

# Assuming gradient_table needs to be constructed if gtab_file is not directly usable

# bvals, bvecs = np.loadtxt(gtab_file[:-5] + '.bvals'), np.loadtxt(gtab_file[:-5] + '.bvecs')  # Adjust filenames
gtab = gradient_table(bvals, bvecs)

(128, 128, 60, 193) (193,) (3, 193) (91, 109, 91)


In [31]:
tensor_model = TensorModel(gtab)
tensor_fit = tensor_model.fit(img)

FA = fractional_anisotropy(tensor_fit.evals)
MD = mean_diffusivity(tensor_fit.evals)
RD = radial_diffusivity(tensor_fit.evals)
AD = axial_diffusivity(tensor_fit.evals)


In [ ]:
labels = atlas  # Assuming atlas data is already loaded correctly
unique_labels = np.unique(labels)[1:]  # Exclude 0, assuming it's background

# Prepare DataFrame to hold features
df = pd.DataFrame(columns=['ROI', 'FA_mean', 'MD_mean', 'RD_mean', 'AD_mean'])

# Loop through each label and calculate mean DTI metrics
for roi in unique_labels:
    mask = labels == roi
    fa_mean = np.mean(FA[mask])
    md_mean = np.mean(MD[mask])
    rd_mean = np.mean(RD[mask])
    ad_mean = np.mean(AD[mask])
    
    # Append the results to the DataFrame
    df = df.append({'ROI': roi, 'FA_mean': fa_mean, 'MD_mean': md_mean, 'RD_mean': rd_mean, 'AD_mean': ad_mean}, ignore_index=True)


In [ ]:
df.to_csv('dti_metrics_by_roi.csv', index=False)

plt.figure(figsize=(12, 10))
plt.title('Mean FA by ROI')
sorted_df = df.sort_values(by='FA_mean', ascending=False)
plt.barh(sorted_df['ROI'][:20], sorted_df['FA_mean'][:20], color='blue')
plt.xlabel('Fractional Anisotropy (FA)')
plt.ylabel('ROI')
plt.show()
